# 01. PyTorch 기초 - Tensor 완전 정복

## 학습 목표
- Tensor의 개념과 NumPy array와의 차이
- Tensor 생성, 연산, 변형
- GPU 사용 및 메모리 관리
- Broadcasting과 Indexing

---

In [1]:
import torch
import numpy as np

print(f"PyTorch version: {torch.__version__}")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

PyTorch version: 2.9.0+cpu
Using device: cpu


## 1. Tensor 생성

**Tensor**: PyTorch의 기본 데이터 구조. N차원 배열.

### NumPy와의 차이
1. **GPU 지원**: Tensor는 GPU에서 연산 가능
2. **Autograd**: 자동 미분 지원 (딥러닝의 핵심!)
3. **최적화**: CUDA, CuDNN 등 GPU 최적화

In [2]:
# 1.1 직접 생성
x = torch.tensor([[1, 2], [3, 4]])
print("Direct creation:")
print(x)
print(f"Shape: {x.shape}, dtype: {x.dtype}, device: {x.device}")

# 1.2 특수 Tensor
zeros = torch.zeros(2, 3)
ones = torch.ones(2, 3)
eyes = torch.eye(3)  # Identity matrix
rand = torch.rand(2, 3)  # Uniform [0, 1)
randn = torch.randn(2, 3)  # Normal(0, 1)

print("\nZeros:")
print(zeros)
print("\nRandom (uniform):")
print(rand)

# 1.3 NumPy에서 변환
arr = np.array([[5, 6], [7, 8]])
from_numpy = torch.from_numpy(arr)
print("\nFrom NumPy:")
print(from_numpy)


Direct creation:
tensor([[1, 2],
        [3, 4]])
Shape: torch.Size([2, 2]), dtype: torch.int64, device: cpu

Zeros:
tensor([[0., 0., 0.],
        [0., 0., 0.]])

Random (uniform):
tensor([[0.5287, 0.0981, 0.3909],
        [0.7114, 0.4565, 0.7059]])

From NumPy:
tensor([[5, 6],
        [7, 8]])


## 2. Tensor 연산

### 2.1 기본 연산 (Element-wise)

In [3]:
a = torch.tensor([1.0, 2.0, 3.0])
b = torch.tensor([4.0, 5.0, 6.0])

# 덧셈
print("a + b =", a + b)
print("torch.add(a, b) =", torch.add(a, b))

# 곱셈 (element-wise)
print("\na * b =", a * b)

# 제곱
print("\na ** 2 =", a ** 2)

# 활성화 함수
print("\ntorch.sigmoid(a) =", torch.sigmoid(a))
print("torch.relu(a) =", torch.relu(a))

a + b = tensor([5., 7., 9.])
torch.add(a, b) = tensor([5., 7., 9.])

a * b = tensor([ 4., 10., 18.])

a ** 2 = tensor([1., 4., 9.])

torch.sigmoid(a) = tensor([0.7311, 0.8808, 0.9526])
torch.relu(a) = tensor([1., 2., 3.])


### 2.2 행렬 연산

In [4]:
A = torch.randn(3, 4)
B = torch.randn(4, 2)

# 행렬 곱 (Matrix multiplication)
C = torch.matmul(A, B)  # 또는 A @ B
print(f"A shape: {A.shape}, B shape: {B.shape}")
print(f"C = A @ B, shape: {C.shape}")

# Transpose
print(f"\nA.T shape: {A.T.shape}")

# Dot product (벡터의 내적)
v1 = torch.tensor([1.0, 2.0, 3.0])
v2 = torch.tensor([4.0, 5.0, 6.0])
dot = torch.dot(v1, v2)
print(f"\nv1 · v2 = {dot}")

A shape: torch.Size([3, 4]), B shape: torch.Size([4, 2])
C = A @ B, shape: torch.Size([3, 2])

A.T shape: torch.Size([4, 3])

v1 · v2 = 32.0


### 2.3 Reduction 연산

In [5]:
x = torch.randn(3, 4)
print("x:")
print(x)

# 전체 합
print(f"\nSum (all): {x.sum()}")

# 차원별 합
print(f"Sum (dim=0): {x.sum(dim=0)}")  # 각 열의 합
print(f"Sum (dim=1): {x.sum(dim=1)}")  # 각 행의 합

# 평균, 최대, 최소
print(f"\nMean: {x.mean()}")
print(f"Max: {x.max()}")
print(f"Argmax (전체): {x.argmax()}")

x:
tensor([[ 0.4099, -0.7446,  0.1351, -0.0488],
        [ 0.0169,  1.3617, -0.2291,  0.3946],
        [-0.0766,  0.0057,  0.1608,  1.8341]])

Sum (all): 3.219841480255127
Sum (dim=0): tensor([0.3503, 0.6228, 0.0668, 2.1800])
Sum (dim=1): tensor([-0.2483,  1.5441,  1.9240])

Mean: 0.26832011342048645
Max: 1.8341292142868042
Argmax (전체): 11


## 3. Tensor 변형 (Reshaping)

**중요**: 딥러닝에서 데이터 크기 변경은 매우 빈번!

In [6]:
x = torch.arange(12)  # [0, 1, 2, ..., 11]
print("Original:", x)

# Reshape
x_reshaped = x.view(3, 4)
print("\nview(3, 4):")
print(x_reshaped)

# -1: 자동 계산
x_auto = x.view(2, -1)  # 2 rows, auto columns
print("\nview(2, -1):")
print(x_auto)

# Flatten
x_flat = x_reshaped.flatten()
print("\nFlattened:", x_flat)

# Squeeze & Unsqueeze (차원 제거/추가)
x = torch.randn(1, 3, 1, 4)
print(f"\nOriginal shape: {x.shape}")
print(f"After squeeze: {x.squeeze().shape}")  # (3, 4)
print(f"After unsqueeze(0): {x.squeeze().unsqueeze(0).shape}")  # (1, 3, 4)

Original: tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

view(3, 4):
tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])

view(2, -1):
tensor([[ 0,  1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10, 11]])

Flattened: tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

Original shape: torch.Size([1, 3, 1, 4])
After squeeze: torch.Size([3, 4])
After unsqueeze(0): torch.Size([1, 3, 4])


## 4. Indexing & Slicing

NumPy와 거의 동일!

In [7]:
x = torch.arange(20).reshape(4, 5)
print("x:")
print(x)

# 단일 요소
print(f"\nx[0, 0] = {x[0, 0]}")

# 행 선택
print(f"\nx[1] = {x[1]}")

# 열 선택
print(f"\nx[:, 2] = {x[:, 2]}")

# Slicing
print(f"\nx[1:3, 2:4] =\n{x[1:3, 2:4]}")

# Boolean indexing
mask = x > 10
print(f"\nx[x > 10] = {x[mask]}")

x:
tensor([[ 0,  1,  2,  3,  4],
        [ 5,  6,  7,  8,  9],
        [10, 11, 12, 13, 14],
        [15, 16, 17, 18, 19]])

x[0, 0] = 0

x[1] = tensor([5, 6, 7, 8, 9])

x[:, 2] = tensor([ 2,  7, 12, 17])

x[1:3, 2:4] =
tensor([[ 7,  8],
        [12, 13]])

x[x > 10] = tensor([11, 12, 13, 14, 15, 16, 17, 18, 19])


## 5. Broadcasting

**Broadcasting**: 크기가 다른 Tensor끼리 연산할 때 자동으로 크기 맞춤

**규칙**:
1. 뒤에서부터 차원 비교
2. 차원이 1이거나 같으면 OK
3. 없는 차원은 1로 간주

In [8]:
# 예제 1: Scalar + Matrix
x = torch.ones(3, 4)
y = 10 
print("x + 10:")
print(x + y)

# 예제 2: Vector + Matrix
x = torch.ones(3, 4)
v = torch.tensor([1, 2, 3, 4])  # (4,)
print("\nx + v (broadcasting):")
print(x + v)  # v가 (3, 4)로 확장됨

# 예제 3: Column vector + Matrix
v_col = torch.tensor([[1], [2], [3]])  # (3, 1)
print("\nx + v_col:")
print(x + v_col)  # v_col이 (3, 4)로 확장

# 시각화
print("\n=== Broadcasting 규칙 ===")
print("x.shape = (3, 4)")
print("v.shape = (4,) → (1, 4) → (3, 4)")
print("v_col.shape = (3, 1) → (3, 4)")

x + 10:
tensor([[11., 11., 11., 11.],
        [11., 11., 11., 11.],
        [11., 11., 11., 11.]])

x + v (broadcasting):
tensor([[2., 3., 4., 5.],
        [2., 3., 4., 5.],
        [2., 3., 4., 5.]])

x + v_col:
tensor([[2., 2., 2., 2.],
        [3., 3., 3., 3.],
        [4., 4., 4., 4.]])

=== Broadcasting 규칙 ===
x.shape = (3, 4)
v.shape = (4,) → (1, 4) → (3, 4)
v_col.shape = (3, 1) → (3, 4)


## 6. GPU 사용

**CPU vs GPU**:
- CPU: 순차 처리에 강함
- GPU: 병렬 처리에 강함 (행렬 연산!)

**주의**: GPU와 CPU 간 데이터 이동은 느림!

In [9]:
# CPU Tensor
x_cpu = torch.randn(3, 3)
print(f"CPU tensor device: {x_cpu.device}")

if torch.cuda.is_available():
    # GPU로 이동
    x_gpu = x_cpu.to('cuda')  # 또는 .cuda()
    print(f"GPU tensor device: {x_gpu.device}")
    
    # GPU에서 직접 생성
    y_gpu = torch.randn(3, 3, device='cuda')
    
    # GPU에서 연산
    z_gpu = x_gpu + y_gpu
    print(f"Result device: {z_gpu.device}")
    
    # CPU로 다시 가져오기
    z_cpu = z_gpu.cpu()  # 또는 .to('cpu')
    print(f"Back to CPU: {z_cpu.device}")
    
    # 속도 비교
    import time
    
    size = 5000
    a_cpu = torch.randn(size, size)
    b_cpu = torch.randn(size, size)
    a_gpu = a_cpu.cuda()
    b_gpu = b_cpu.cuda()
    
    # CPU
    start = time.time()
    c_cpu = a_cpu @ b_cpu
    print(f"\nCPU time: {time.time() - start:.4f}s")
    
    # GPU
    torch.cuda.synchronize()  # GPU 연산 완료 대기
    start = time.time()
    c_gpu = a_gpu @ b_gpu
    torch.cuda.synchronize()
    print(f"GPU time: {time.time() - start:.4f}s")
    
else:
    print("GPU not available")

CPU tensor device: cpu
GPU not available


## 7. In-place 연산

**In-place**: 원본 Tensor를 직접 수정 (메모리 절약)

**주의**: Autograd와 함께 사용 시 조심!

In [10]:
x = torch.tensor([1.0, 2.0, 3.0])
print(f"Original x: {x}")
print(f"ID: {id(x)}")

# 일반 연산 (새 Tensor 생성)
y = x.add(10)
print(f"\nAfter y = x.add(10):")
print(f"x: {x}")
print(f"y: {y}")
print(f"x ID: {id(x)}, y ID: {id(y)}")

# In-place 연산 (x 직접 수정)
x.add_(10)  # '_' suffix = in-place
print(f"\nAfter x.add_(10):")
print(f"x: {x}")
print(f"ID: {id(x)}")

# 다른 in-place 연산들
# x.mul_(2)    # x *= 2
# x.zero_()    # x = 0
# x.fill_(5)   # x = 5

Original x: tensor([1., 2., 3.])
ID: 132029683922608

After y = x.add(10):
x: tensor([1., 2., 3.])
y: tensor([11., 12., 13.])
x ID: 132029683922608, y ID: 132029683919888

After x.add_(10):
x: tensor([11., 12., 13.])
ID: 132029683922608


## 8. 연습문제

### 문제 1: Batch Matrix Multiplication
배치 크기 32, 입력 차원 10, 출력 차원 5인 선형 변환을 구현하시오.

$$Y = XW^T + b$$

where X: (32, 10), W: (5, 10), b: (5,)

In [11]:
# 문제 1 풀이
batch_size = 32
input_dim = 10
output_dim = 5

X = torch.randn(batch_size, input_dim)
W = torch.randn(output_dim, input_dim)
b = torch.randn(output_dim)

# Y = XW^T + b
Y = X @ W.T + b  # Broadcasting: b (5,) → (32, 5)

print(f"X shape: {X.shape}")
print(f"W shape: {W.shape}")
print(f"b shape: {b.shape}")
print(f"Y shape: {Y.shape}")
print(f"\nExpected: (32, 5)")

X shape: torch.Size([32, 10])
W shape: torch.Size([5, 10])
b shape: torch.Size([5])
Y shape: torch.Size([32, 5])

Expected: (32, 5)


### 문제 2: Softmax 구현

$$\text{softmax}(x_i) = \frac{e^{x_i}}{\sum_j e^{x_j}}$$

In [12]:
# 문제 2 풀이
def my_softmax(x, dim=-1):
    """Numerically stable softmax"""
    # Subtract max for numerical stability
    x_max = x.max(dim=dim, keepdim=True)[0]
    x_exp = torch.exp(x - x_max)
    return x_exp / x_exp.sum(dim=dim, keepdim=True)

# Test
logits = torch.randn(4, 10)  # 배치 4, 클래스 10
probs_my = my_softmax(logits, dim=1)
probs_torch = torch.softmax(logits, dim=1)

print("My softmax:")
print(probs_my[0])
print(f"Sum: {probs_my[0].sum()}")

print("\nPyTorch softmax:")
print(probs_torch[0])

print(f"\nDifference: {(probs_my - probs_torch).abs().max()}")

My softmax:
tensor([0.0963, 0.1847, 0.0423, 0.0282, 0.1567, 0.0596, 0.1344, 0.0879, 0.1291,
        0.0810])
Sum: 1.0000001192092896

PyTorch softmax:
tensor([0.0963, 0.1847, 0.0423, 0.0282, 0.1567, 0.0596, 0.1344, 0.0879, 0.1291,
        0.0810])

Difference: 1.4901161193847656e-08


## 9. 요약

### 핵심 개념
1. **Tensor**: PyTorch의 기본 데이터 구조, N차원 배열
2. **GPU 지원**: `.to(device)` or `.cuda()`
3. **Broadcasting**: 자동 크기 맞춤
4. **In-place**: `_` suffix (메모리 절약, 주의 필요)

### 주요 함수
- **생성**: `torch.tensor()`, `torch.zeros()`, `torch.randn()`
- **연산**: `+`, `*`, `@`, `.matmul()`, `.dot()`
- **변형**: `.view()`, `.reshape()`, `.transpose()`, `.squeeze()`
- **Reduction**: `.sum()`, `.mean()`, `.max()`, `.argmax()`

### 다음 단계
- `02_autograd_and_gradients.ipynb` - 자동 미분의 핵심!